In [ ]:
import pandas as pd
import os

In [ ]:
caminho = os.path.join('..', '..', 'db_softtek') 
path_arq = os.path.abspath(caminho) 
db_analitico = '\\cubo_anl\\tbl_full_analitica.xlsx'

In [ ]:
# Carrega tabela de demandas:
tbl_anl = pd.read_excel(path_arq+db_analitico, dtype='string').drop_duplicates()

In [ ]:
#funções genéricas:


#Função anomes:
def calcul_anomes(df,campo_data):
    if campo_data == 'DT_ABERTURA_CHAMADO':
        nova_coluna = 'ANOMES_ABERTURA'
        df[nova_coluna] = df['DT_ABERTURA_CHAMADO'].astype(str).str[0:4] + df['DT_ABERTURA_CHAMADO'].astype(str).str[5:7]
    else:
        nova_coluna = 'ANOMES_STATUS'
        df[nova_coluna] = df['DT_STATUS_CHAMADO'].astype(str).str[0:4] + df['DT_STATUS_CHAMADO'].astype(str).str[5:7]

    return df

#Função calculo Leadtime de atendimento:
def coluna_leadtime(dias):
    if dias == '0 days':
        return 'D0'
    if dias == '1 days':
        return 'D1'
    elif dias == '2 days':
        return 'D2'
    elif dias == '3 days':
        return 'D3'
    elif dias == '4 days':
        return 'D4'
    else:
        return 'D5+'

In [ ]:
#Campo de data abertura do chamado:
tbl_anl['DT_ABERTURA_CHAMADO'] = tbl_anl['DT_ABERTURA_CHAMADO'].astype('datetime64[ns]').dt.strftime("%Y-%m-%d")

#Campo de data status do Chamado:

tbl_anl['DT_STATUS_CHAMADO'] = tbl_anl['EXERCICIO']+"-"+tbl_anl['PERIODO'].str.zfill(2)+"-"+tbl_anl['DIA'].str.zfill(2)
tbl_anl['DT_STATUS_CHAMADO'] = tbl_anl['DT_STATUS_CHAMADO'].astype('datetime64[ns]').dt.strftime("%Y-%m-%d")


# Calcula anomes 
tbl_anl = calcul_anomes(tbl_anl,'DT_ABERTURA_CHAMADO')
tbl_anl = calcul_anomes(tbl_anl,'DT_STATUS_CHAMADO')
tbl_anl = tbl_anl.sort_values(['CHAMADO','DT_ABERTURA_CHAMADO','DT_STATUS_CHAMADO'])

# Criando campos de LeadTime:
tbl_anl['LEADTIME_CHAMADO'] = tbl_anl['DT_STATUS_CHAMADO'].astype('datetime64[ns]') - tbl_anl['DT_ABERTURA_CHAMADO'].astype('datetime64[ns]')
tbl_anl['LEADTIME_CALCULADO'] = tbl_anl['LEADTIME_CHAMADO'].astype(str).apply(coluna_leadtime)




In [ ]:
# Indicador de volume de demandas distintas:
tbl_anl_abert = tbl_anl.drop_duplicates(subset=['CHAMADO','DIA','CONSULTOR'], keep='first')
tbl_anl_abert = tbl_anl[['ANOMES_ABERTURA','SENIORIDADE','MODULO_CustosNSULTOR','CHAMADO']].groupby(['ANOMES_ABERTURA','SENIORIDADE','MODULO_CustosNSULTOR']).agg({'CHAMADO':'count'}).reset_index()
tbl_anl_abert['INDICADOR'] = 'ABERTURAS'

tbl_anl_abert.rename(columns={'MODULO_CustosNSULTOR':'ATRIBUTO','ANOMES_ABERTURA':'ANOMES_REF',
                              'SENIORIDADE':'ATRIBUTO2'}, inplace=True)
tbl_anl_abert.head(5)

In [ ]:
tbl_anl = tbl_anl.drop_duplicates(subset=['CHAMADO','DIA','CONSULTOR'], keep='last')

In [ ]:
tbl_anl_pendentes = tbl_anl[tbl_anl['STATUS_CHAMADO'] !='Closed']
tbl_anl_pendentes = tbl_anl_pendentes[['ANOMES_ABERTURA','SENIORIDADE','COMPLEXIDADE','CHAMADO']].groupby(['ANOMES_ABERTURA','SENIORIDADE','COMPLEXIDADE']).agg({'CHAMADO':'count'}).reset_index()
tbl_anl_pendentes['INDICADOR'] = 'PENDENTE'
tbl_anl_pendentes.rename(columns={'COMPLEXIDADE':'ATRIBUTO','ANOMES_ABERTURA':'ANOMES_REF',
                              'SENIORIDADE':'ATRIBUTO2'}, inplace=True)
tbl_anl_pendentes.head(5)

In [ ]:
tbl_anl_encerr = tbl_anl[tbl_anl['STATUS_CHAMADO'] =='Closed']
tbl_anl_encerr = tbl_anl_encerr[['ANOMES_STATUS','SENIORIDADE','MODULO_CustosNSULTOR','CHAMADO']].groupby(['ANOMES_STATUS','SENIORIDADE','MODULO_CustosNSULTOR']).agg({'CHAMADO':'count'}).reset_index()
tbl_anl_encerr['INDICADOR'] = 'ENCERRAMENTO'
tbl_anl_encerr.rename(columns={'MODULO_CustosNSULTOR':'ATRIBUTO','ANOMES_STATUS':'ANOMES_REF',
                              'SENIORIDADE':'ATRIBUTO2'}, inplace=True)
tbl_anl_encerr.head(5)

In [ ]:
tbl_anl_long = tbl_anl[tbl_anl['STATUS_CHAMADO'] =='Closed']
tbl_anl_long = tbl_anl_long[['ANOMES_STATUS','LEADTIME_CALCULADO','MODULO_CustosNSULTOR','CHAMADO']].groupby(['ANOMES_STATUS','LEADTIME_CALCULADO','MODULO_CustosNSULTOR']).agg({'CHAMADO':'count'}).reset_index()
tbl_anl_long['INDICADOR'] = 'LEADTIME'
tbl_anl_long.rename(columns={'MODULO_CustosNSULTOR':'ATRIBUTO','ANOMES_STATUS':'ANOMES_REF',
                              'LEADTIME_CALCULADO':'ATRIBUTO2'}, inplace=True)
tbl_anl_long.head(50)

In [ ]:
tbl_anl_complex = tbl_anl[tbl_anl['STATUS_CHAMADO'] =='Closed']
tbl_anl_complex = tbl_anl_complex[['ANOMES_STATUS','SENIORIDADE','COMPLEXIDADE','CHAMADO']].groupby(['ANOMES_STATUS','SENIORIDADE','COMPLEXIDADE']).agg({'CHAMADO':'count'}).reset_index()
tbl_anl_complex['INDICADOR'] = 'COMPLEXIDADE_VOL'
tbl_anl_complex.rename(columns={'COMPLEXIDADE':'ATRIBUTO','ANOMES_STATUS':'ANOMES_REF',
                              'SENIORIDADE':'ATRIBUTO2'}, inplace=True)
tbl_anl_complex.head(5)

In [ ]:
tbl_anl_cargo_VOL = tbl_anl[['ANOMES_STATUS','SENIORIDADE','CHAMADO','MODULO_CustosNSULTOR']].groupby(['ANOMES_STATUS','SENIORIDADE','MODULO_CustosNSULTOR']).agg({'CHAMADO':'count'}).reset_index()
tbl_anl_cargo_VOL.rename(columns={'MODULO_CustosNSULTOR':'ATRIBUTO','ANOMES_STATUS':'ANOMES_REF',
                              'LEADTIME_CALCULADO':'ATRIBUTO2'}, inplace=True)
tbl_anl_cargo_VOL['INDICADOR'] = 'VOLUMETRIA CARGO'


tbl_anl_cargo_tma = tbl_anl[['ANOMES_STATUS','SENIORIDADE','COMPLEXIDADE','HORAS','VALOR_AT','CONSULTOR']].drop_duplicates()
tbl_anl_cargo_tma['HORAS'] = round(tbl_anl_cargo_tma['HORAS'].astype('float'),2)
tbl_anl_cargo_tma['VALOR_AT'] = round(tbl_anl_cargo_tma['VALOR_AT'].astype('float'),2)

tbl_anl_cargo_tma = tbl_anl_cargo_tma.groupby(['ANOMES_STATUS','SENIORIDADE','COMPLEXIDADE']).agg({'HORAS':'mean', 'VALOR_AT':'mean'}).reset_index()
tbl_anl_cargo_tma.head(5)

In [ ]:
path_arq

In [ ]:
cubo_1 = pd.concat([tbl_anl_abert,tbl_anl_encerr,tbl_anl_long,tbl_anl_pendentes,tbl_anl_complex,tbl_anl_cargo_VOL], axis=0)
cubo_1.to_excel(path_arq+'\\cubo_anl\\cubo1_consolidado.xlsx', index=False)

In [ ]:
tbl_anl_cargo_tma.to_excel(path_arq+'\\cubo_anl\\cubo2_consolidado.xlsx', index=False)